In [1]:
 # data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#import nltk
import nltk
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

#stop-words
stop_words_nl=set(nltk.corpus.stopwords.words('dutch'))
stop_words_en=set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bertram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/bertram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/bertram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/bertram/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to /home/bertram/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [1]:
import os
import re

texts = []

for file in os.listdir("Translation/translations_txt/"):
    if re.match(r"\d*\.txt", file):
        with open(f"Translation/translations_txt/{file}") as text:
            texts.append(text.read())

print(len(texts))
print(texts[0])

768
Publication: 2020-04-24Numac: 2020030601federal Public Service Home Affairs27 February 2019.-Act amending the Royal Decree No. 20 of 20 July 1970 establishing the rates of the added value and to classify the goods and the Services at those rates with regard to the VAT rate for the delivery of plants and flowers in landscaping and maintenance. - German translation below is the German translation of the law of 27 February 2019 amending Royal Decree No. 20 of 20 July 1970 set the rates of the value added tax and to classify the goods and the services that Rates with regard to the VAT rate for the delivery of plants and flowers in the garden and maintenance (Belgian Official Gazette of 14 March 2019). This translation was drawn up by the Central Service for German translation in Malmedy.föderaler Öffentlicher Service Finanzen27.Febrar 2019-Gesetz Zur Abänderung Hinsichtlich des Mehrwertsteuersatzes für die Lieferung von Pflanzenund Blumen Bei der Gestaltung und Pflege von Gärten des Kö

In [26]:
import spacy
import nl_core_news_sm

nlp = nl_core_news_sm.load()
#en_words = set(nltk.corpus.words.words("dutch"))
overly_abundant = ["artikel", "besluit", "publicatie"]
months = ["januari", "februari", "maart", "april", "mei", "juni", "juli", "augustus", "september", "november", "december"]

def lemmatizer(texts):
    texts = [re.sub(r'[\W\d]', " ", text) for text in texts]
    docs = nlp.pipe(texts)
    cleaned_lemmas = [[t.lemma_ for t in doc if t.lemma_ not in stop_words_nl 
                        #and t.lemma_ in en_words
                        and t.lemma_ not in stop_words_en
                        and t.lemma_.lower() not in overly_abundant
                        and t.lemma_.lower() not in months
                        and not re.match(r".*\d.*", t.lemma_)] for doc in docs]

    return cleaned_lemmas

In [5]:
cleaned = []
parts = int(len(texts)/20) + 1
for part in range(parts):
    print(f"Batch {part + 1} of {parts}")
    cleaned.append(lemmatizer(texts[20*part:20*(part+1)]))
#lemmatizer(texts)
print(len(cleaned))
print(cleaned[0][0])

Batch 1 of 38
Batch 2 of 38
Batch 3 of 38
Batch 4 of 38
Batch 5 of 38
Batch 6 of 38
Batch 7 of 38
Batch 8 of 38
Batch 9 of 38
Batch 10 of 38
Batch 11 of 38
Batch 12 of 38
Batch 13 of 38
Batch 14 of 38
Batch 15 of 38
Batch 16 of 38
Batch 17 of 38
Batch 18 of 38
Batch 19 of 38
Batch 20 of 38
Batch 21 of 38
Batch 22 of 38
Batch 23 of 38
Batch 24 of 38
Batch 25 of 38
Batch 26 of 38
Batch 27 of 38
Batch 28 of 38
Batch 29 of 38
Batch 30 of 38
Batch 31 of 38
Batch 32 of 38
Batch 33 of 38
Batch 34 of 38
Batch 35 of 38
Batch 36 of 38
Batch 37 of 38
Batch 38 of 38
38
[':', ':', 'FEDERALE', 'OVERHEIDSDIENST', 'BINNENLANDSE', 'ZAKEN', ' ', ' ', '-', 'wet', 'wijziging', 'koninklijk', 'nummer', ' ', 'juli', 'vaststelling', 'tarief', 'belasting', 'toegevoegde', 'waarde', 'indeling', 'goed', 'dienst', 'tarief', 'bepalen', 'betreffen', ' ', '-', 'Duits', 'vertaling', ' ', 'hierna', 'volgen', 'tekst', ' ', 'Duits', 'vertaling', 'wet', 'wijziging', 'koninklijk', 'nummer', ' ', ' ', 'juli', 'vaststelling'

In [2]:
from preprocessing import lemmatization

lemmatized = lemmatization.lemmatizer(texts, batchsize=20)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bertram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/bertram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/bertram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/bertram/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to /home/bertram/nltk_data...
[nltk_data]   Package words is already up-to-date!


v2.4: Batch 1 of 39
Batch 1 done!
v2.4: Batch 2 of 39
Batch 2 done!
v2.4: Batch 3 of 39
Batch 3 done!
v2.4: Batch 4 of 39
Batch 4 done!
v2.4: Batch 5 of 39
Batch 5 done!
v2.4: Batch 6 of 39
Batch 6 done!
v2.4: Batch 7 of 39
Batch 7 done!
v2.4: Batch 8 of 39
Batch 8 done!
v2.4: Batch 9 of 39
Batch 9 done!
v2.4: Batch 10 of 39
Batch 10 done!
v2.4: Batch 11 of 39
Batch 11 done!
v2.4: Batch 12 of 39
Batch 12 done!
v2.4: Batch 13 of 39
Batch 13 done!
v2.4: Batch 14 of 39
Batch 14 done!
v2.4: Batch 15 of 39
Batch 15 done!
v2.4: Batch 16 of 39
Batch 16 done!
v2.4: Batch 17 of 39
Batch 17 done!
v2.4: Batch 18 of 39
Batch 18 done!
v2.4: Batch 19 of 39
Batch 19 done!
v2.4: Batch 20 of 39
Batch 20 done!
v2.4: Batch 21 of 39
Batch 21 done!
v2.4: Batch 22 of 39
Batch 22 done!
v2.4: Batch 23 of 39
Batch 23 done!
v2.4: Batch 24 of 39
Batch 24 done!
v2.4: Batch 25 of 39
Batch 25 done!
v2.4: Batch 26 of 39
Batch 26 done!
v2.4: Batch 27 of 39
Batch 27 done!
v2.4: Batch 28 of 39
Batch 28 done!
v2.4: Batc

In [3]:
print(len(lemmatized))
print(lemmatized[0])

768
federal amend establish rate value good rate regard rate delivery plant flower maintenance german translation german translation amend rate value good service regard rate delivery plant flower garden maintenance official translation draw german translation allen fell hear wird nach nach federal amend establish rate value good service rate regard certain german translation german translation amend establish rate value good service regard certain official translation draw german translation allen wird stick serve video instrument wird treat knob title sind full work ental seine honorary shine describe advertising sheet brochure price list course sheet maritime information notarial report timetable specialized commercial think game publish name industrial finance commercial company even contain text illustration general interest without direct advertising whose main purpose search industrial finance trade business continue expand represent advertising material company deliver impact i

In [6]:
print(len(cleaned[0]))

20


In [5]:
vect = TfidfVectorizer(stop_words=stop_words_en,max_features=1000)

In [8]:
cleaned_joined = []
    
[[cleaned_joined.append(" ".join(words)) for words in batch] for batch in cleaned]
print(len(cleaned_joined))

760


In [6]:
vectorised = vect.fit_transform(lemmatized)
vect.get_feature_names_out()

array(['aanbeveling', 'aanbrengen', 'aandeel', 'aangelegenheid',
       'aangereken', 'aangetekend', 'aangezien', 'aangifte', 'aanhef',
       'aanleiding', 'aanmerking', 'aannemen', 'aanpassen', 'aanpassing',
       'aanslagjaar', 'aantal', 'aanvraag', 'aanvrag', 'aanvrager',
       'aanvullen', 'aanwezig', 'aanwijzen', 'aanzienlijk', 'aard',
       'abgeändren', 'absatz', 'abänderung', 'acht', 'achten', 'actie',
       'actief', 'activiteit', 'activités', 'administratie',
       'administratief', 'adres', 'advies', 'afdeling', 'afnemer',
       'afsluiten', 'afstand', 'afwijking', 'agentschap', 'akkoord',
       'akkoordbevinding', 'akte', 'aktivität', 'aldus', 'algemeen',
       'algemene', 'alleen', 'alsook', 'alter', 'alternatief', 'ambt',
       'ambtenaar', 'ambtenarenzaken', 'ander', 'anderzijds', 'anwendbar',
       'anwendung', 'arbeid', 'arbeidsovereenkomst', 'article',
       'artikeln', 'arts', 'attest', 'auch', 'aufgehoben', 'aufgrund',
       'august', 'ausbreitung', 'au

In [7]:
idf=vect.idf_
dd=dict(zip(vect.get_feature_names_out(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0:10])

['minister', 'volgen', 'betreffen', 'overheidsdienst', 'covid', 'houden', 'brussel', 'belgisch', 'toepassing', 'geven']


## LDA

In [12]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 

lda_top=lda_model.fit_transform(vectorised)

In [13]:
# most important words for each topic
vocab = vect.get_feature_names()

for i, comp in enumerate(lda_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
lijst overeenkomst fod niveau ondersteuning en af_sluiten resultaat jaar via 

Topic 1: 
collectief paritair arbeidsovereenkomst recht overeenkomstig belgië autoriteit werkloosheid belastbaar instelling 

Topic 2: 
nace bel koninklijk code automatisch lijst gelet wet onroerend bekrachtigen 

Topic 3: 
autoriteit akkoord belgië bevoegd overeenkomst onderling verlengen mei overleg belasting 

Topic 4: 
verstrekking test patiënt vermelden verzekering extern persoon agentschap mei decreet 

Topic 5: 
vertaling duits identificatienummer bericht wetboek centraal fiscaal inlichting op_maken toegevoegde 

Topic 6: 
regering gelet lid minister wet decreet brussels overwegende art gemeenschap 

Topic 7: 
lening vertaling und maart gelet koninklijk juli art duits overwegende 

Topic 8: 
und des von vom das wird für den auf zur 

Topic 9: 
aanslagjaar belasting wetboek bedoelen gelet overwegende wet lid premie koninklijk 



/home/bertram/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## BERTopic

In [50]:
from bertopic import BERTopic

topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(lemmatized)

Batches:   0%|          | 0/24 [00:00<?, ?it/s]

2022-06-14 12:51:03,635 - BERTopic - Transformed documents to Embeddings
2022-06-14 12:51:07,079 - BERTopic - Reduced dimensionality
2022-06-14 12:51:07,200 - BERTopic - Clustered reduced embeddings


In [146]:
topic_model.transform(lemmatized[2])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-06-14 16:13:26,334 - BERTopic - Reduced dimensionality
2022-06-14 16:13:26,341 - BERTopic - Calculated probabilities with HDBSCAN
2022-06-14 16:13:26,342 - BERTopic - Predicted clusters


([0],
 array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]))

In [44]:
new_topics, new_probs = topic_model.reduce_topics(lemmatized, topics, probs, nr_topics=16)

2022-06-14 12:47:27,771 - BERTopic - Reduced number of topics from 27 to 17


In [19]:
lemmatized[2]

'federal amend regard exemption transfer leasehold german translation translation amend term exemption transfer leasehold official translation draw allen become road state'

In [53]:
topic_model.visualize_barchart(top_n_topics=50, n_words=9)

In [24]:
topic_model.visualize_heatmap()

In [155]:
similar_topics, similarity = topic_model.find_topics("translation", top_n=50); 
topic_similarity_keyword = dict(zip(similar_topics, similarity))

In [156]:
topic_array = []
for index in range(len(topic_similarity_keyword) - 1):
    topic_array.append(topic_similarity_keyword[index])

similarity = []
cosine_similar_articles = []

for elem in test[1]:
    similarity.append(cosine_similarity([topic_array, elem]))

for index in range(len(similarity)):
    if similarity[index][0][1] > 0.85:
        cosine_similar_articles.append(index)
        print(index)
        print(similarity[index][0][1])

3
0.9458720284751353
29
0.9750651622640384
32
0.9914793795232126
34
0.897051776144257
39
0.9564433890890432
42
0.9914793804016981
51
0.9448166080743549
67
0.8840906956789606
82
0.8866165487928706
100
0.9305802616472377
112
0.896910229232243
113
0.956441709947086
115
0.8866587640513315
118
0.8804690305603463
119
0.8788375362381347
169
0.8971573267044342
178
0.8951654580841307
181
0.9473257878129429
187
0.9463038372919015
198
0.8776637913084683
199
0.9467720215272711
205
0.9750804120422908
212
0.8762312813766764
214
0.898189055907766
215
0.9624110478338128
245
0.8964355147149264
246
0.9318515551144153
249
0.9564427990002258
251
0.8930587905817934
265
0.9305540879876917
267
0.8826466405193379
269
0.8758251170361784
271
0.9306119363262321
277
0.9306218565414263
284
0.9564423123030824
309
0.991479386295748
322
0.9624372020368045
335
0.9463121229485264
354
0.8793743926683191
369
0.991479412322639
376
0.863838679064732
386
0.9564420041699369
388
0.8971365627606419
400
0.8574466387338697
424
0

In [136]:
print(texts[32])

Publication: 2020-01-24numac: 2020010214ministerie of the French Community 20 December 2019.-Ministerial Decree approval of the reference file of the "Bachelor specialization: tax sciences" department (code 71705S32D2) ranked in the area of ​​the economic and management sciences of the higher Education for social promotion of the short typing minister of Education for Social Promotion, given the laws on the assignment of the academic guards and the program of the university exams, coordinated in the decision of the Regent of December 3, 1949, in particular Article 6, amended by Article 124 of the Decree of the Fransguorschap of 16 April 1991 on the organization of social promotion education; in view of the law of 29 May 1959 amending some provisions of the education legislation, as amended; having regard to the decree of the French Community of 16 April 1991 Organizing the Education Presocial P Romotie, Articles 47, 71, 74, 75 and 137; Having regard to the decision of the Execution Eva

In [111]:
repr_docs = topic_model.get_representative_docs()
for topic in similar_topics:
    if topic == -1:
        continue
    print("#############")
    print(repr_docs[topic])


#############
['federal record model return form resident company association year greeting amend program predominantly insert stipulate subject resident company association must submit return electronic unless case person authorize submit declaration behalf require mean comply obligation view state regard urgent possibility company association year must soon possible order delay location collection decision must therefore make urgently nomination decide decide model declaration form resident company association year annex minister responsible finance implementation brussels consult table image', 'federal record model corporation assessment year king hereby amend insert stipulate taxpayer award company curse declaration must submit electronic unless appropriate person authorize submit declaration behalf necessary resource meet obligation view state view urgent necessity delay decision urgently take nomination decide model declaration form corporation year annex minister power finance r

In [42]:
freq = topic_model.get_topic_info(); freq

,Topic,Count,Name
0,-1,134,-1_measure_paragraph_refer_decision
1,0,68,0_brussels_measure_competent_authority
2,1,54,1_flemish_company_word_vaccination
3,2,52,2_education_student_school_year
4,3,42,3_walloon_decision_employment_subsidy
5,4,37,4_federal_table_instead_image
6,5,34,5_contact_datum_person_personal
7,6,33,6_investment_collective_institution_fire
8,7,24,7_agreement_authority_competent_extend
9,8,24,8_patient_video_benefit_communication


In [11]:
topic_model.get_topic(0)

[('brussels', 0.029625432825213748),
 ('hoofdstedelijk', 0.026526646651979926),
 ('bedoelen', 0.02593321945991014),
 ('wetboek', 0.02283792510914578),
 ('betreffen', 0.01805588518716609),
 ('goed', 0.017784220684189382),
 ('regering', 0.016417636905230376),
 ('woord', 0.016380647230782572),
 ('koninklijk', 0.01631416467711345),
 ('vervangen', 0.016027128035943736)]

In [21]:
topic_model.visualize_topics()

In [24]:
topic_model.visualize_topics()

In [22]:
topic_model.visualize_distribution(probs[200], min_probability=0.015)

In [14]:
topic_model.visualize_distribution(probs[200], min_probability=0.015)

In [23]:
topic_model.visualize_hierarchy(top_n_topics=50)

In [68]:
predictions = []

In [69]:
for lemma in lemmatized:
    predictions.append(topic_model.transform(lemma)[1])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-06-14 14:28:34,142 - BERTopic - Reduced dimensionality
2022-06-14 14:28:34,150 - BERTopic - Calculated probabilities with HDBSCAN
2022-06-14 14:28:34,151 - BERTopic - Predicted clusters


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-06-14 14:28:37,366 - BERTopic - Reduced dimensionality
2022-06-14 14:28:37,372 - BERTopic - Calculated probabilities with HDBSCAN
2022-06-14 14:28:37,373 - BERTopic - Predicted clusters


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

2022-06-14 14:30:58,827 - BERTopic - Reduced dimensionality
2022-06-14 14:30:58,833 - BERTopic - Calculated probabilities with HDBSCAN
2022-06-14 14:30:58,835 - BERTopic - Predicted clusters


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-06-14 14:31:02,274 - BERTopic - Reduced dimensionality
2022-06-14 14:31:02,281 - BERTopic - Calculated probabilities with HDBSCAN
2022-06-14 14:31:02,282 - BERTopic - Predicted clusters


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [112]:
test = topic_model.transform(lemmatized)

Batches:   0%|          | 0/24 [00:00<?, ?it/s]

2022-06-14 15:46:38,914 - BERTopic - Reduced dimensionality
2022-06-14 15:46:39,229 - BERTopic - Calculated probabilities with HDBSCAN
2022-06-14 15:46:39,229 - BERTopic - Predicted clusters


In [148]:
topic_model.transform(lemmatized[173])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-06-14 16:18:16,949 - BERTopic - Reduced dimensionality
2022-06-14 16:18:16,955 - BERTopic - Calculated probabilities with HDBSCAN
2022-06-14 16:18:16,957 - BERTopic - Predicted clusters


([9],
 array([[0.00808486, 0.00508673, 0.01187316, 0.00430128, 0.02427046,
         0.00613511, 0.00774189, 0.01194495, 0.0194754 , 0.06774697,
         0.00560283, 0.0358245 , 0.02015908, 0.0099457 , 0.02091375,
         0.00900728, 0.00539973, 0.00520141, 0.00556701, 0.00752297,
         0.00563233, 0.02010567, 0.00934813, 0.00915723, 0.00551181,
         0.0195499 , 0.00700891, 0.0098605 , 0.0039424 , 0.00529484,
         0.00708724, 0.00688818]]))

In [74]:
len(test[0])

768

In [75]:
test[1]

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00571362, 0.00654805, 0.0056631 , ..., 0.00499942, 0.00536031,
        0.0093705 ],
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00122212, 0.00144667, 0.00127874, ..., 0.00130514, 0.00123531,
        0.00189243],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [152]:
from sklearn.metrics.pairwise import cosine_similarity

test_topic, test_prob = topic_model.transform(lemmatized[3])

similar_articles = []

if test_prob[0][test_topic[0]] >= 0.65:
    print("Hello")
    for index in range(len(test[1])):
        if test[1][index][test_topic[0]] >= 0.65:
            similar_articles.append(index)
    
print(similar_articles)

similarity = []
cosine_similar_articles = []

for elem in test[1]:
    similarity.append(cosine_similarity([test_prob[0], elem]))

for index in range(len(similarity)):
    if similarity[index][0][1] > 0.85:
        cosine_similar_articles.append(index)
        print(index)
        print(similarity[index][0][1])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-06-14 16:21:27,537 - BERTopic - Reduced dimensionality
2022-06-14 16:21:27,543 - BERTopic - Calculated probabilities with HDBSCAN
2022-06-14 16:21:27,545 - BERTopic - Predicted clusters


[]
3
0.9999432955523794
29
0.9435220414991018
32
0.9398405194634494
34
0.9641432294654736
39
0.9220638978152648
42
0.9398405215564402
51
0.9998963826572523
100
0.9374077765601522
112
0.9640880929316995
113
0.9220625464571216
169
0.9641839793787189
181
0.9999853678292759
187
0.9999585681215313
199
0.9999724959788837
205
0.9435183055248852
214
0.9645852541842441
215
0.8888520182760031
246
0.9384400133181623
249
0.9220634419875672
265
0.9373939912280334
271
0.9374416353645533
277
0.9374477919835326
284
0.922063019706535
309
0.9398405506594851
322
0.8888817590767168
335
0.9999588388133762
369
0.9398406889407811
386
0.9220627801003491
388
0.9641764469952385
452
0.9999426902034009
460
0.9641761278353412
468
0.9644676744113955
476
0.9398429857987519
485
0.9999821991295128
506
0.9435224623851403
514
0.8891524974817958
542
0.8884291851052389
565
0.9641887719168365
567
0.9435224314885966
569
0.9435223929801059
571
0.938152964792644
573
0.9398407495625186
609
0.9999867402863306
633
0.999957594813

In [153]:
print(similar_articles)
print(cosine_similar_articles)

[]
[3, 29, 32, 34, 39, 42, 51, 100, 112, 113, 169, 181, 187, 199, 205, 214, 215, 246, 249, 265, 271, 277, 284, 309, 322, 335, 369, 386, 388, 452, 460, 468, 476, 485, 506, 514, 542, 565, 567, 569, 571, 573, 609, 633, 671, 678, 693, 699, 701, 709]


In [157]:
print(cosine_similar_articles)

[3, 29, 32, 34, 39, 42, 51, 67, 82, 100, 112, 113, 115, 118, 119, 169, 178, 181, 187, 198, 199, 205, 212, 214, 215, 245, 246, 249, 251, 265, 267, 269, 271, 277, 284, 309, 322, 335, 354, 369, 376, 386, 388, 400, 424, 452, 460, 461, 466, 468, 476, 477, 485, 488, 506, 514, 516, 517, 522, 527, 540, 542, 565, 567, 569, 571, 573, 581, 609, 617, 625, 633, 671, 678, 680, 684, 693, 699, 701, 705, 709, 714, 736, 742, 749, 753]


In [154]:
print(texts[67])

Publication: 2020-07-01NUMAC: 2020041960Federal Public Service Financial administration for policy expertise and support. - Regulation service. - Agreement for Inlanding Consultation between the competent authorities of France and Belgium with regard to the frontier workers' context of the fight against the distribution of COVID-19 Overweight that France concluded an agreement in mutual consultation on 15 May 2020 (Belgian Official Gazette of 26 May2020, Edition 1, page 37837) Based on Article 24 of the Agreement between France and Belgium of 10 March 1964 to prevent double taxation and regulating mutual administration legal assistance on income taxes, as amended by the Avenanten of 15 February1971, of 8 February 1999 , of December 12, 2008 and from 7 July 2009; the competent authorities of France and Belgium have agreed: 1. The application of the agreement in mutual consultation of 15 May 2020 will be extended until 31 August 2020.2. It has been agreed that for the application of para

In [106]:
for article in similar_articles:
    print("#################")
    print(texts[article])

#################
Publication: 2021-04-20Numac: 2021020770federal Public Service Home Affairs22 April 2019.-Act amending the Code of the Income Begs 1992 with regard to the tax exemption for the transfer of leasehold. - German translation below is the Deduitse translation of the law of 22 April 2019 amending the Income Tax Code in terms of the tax exemption for the transfer of leasehold (Belgian Official Gazette of 29Pril 2019, ERR. Of 9 May 2019). This translation has been drawn up by the Central Service for Gands Version in Malmedy.federaler Öffentlicher Service Finanzen22. April2019 - Gesetz Zur Abänderung des Einkommensteuergesetzbuches 1992 Hinsichtlich der Steuerbe -Freiung fürdie abtretung der Erbpachtphilippe, Konig der Belgier, Allen Gegenwartigen Und Zukunften, UNSER SGUESTEMEN HATEMENTEMEN, FOLEMENTEMEN HATEMENTEMEN - FOLINE FOLENMENTEMEN, FOLINEMENTEMEN, FOLINEMENTEMEN, FOLYEMENTEMEN. der Verfassung Erwähnte Angelegenheit.art.art.2 - Article 12 des Einkommensteuergesetzbuch